In [1]:
import os
import datetime

import IPython
import IPython.display
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
import tensorflow as tf

/tmp/ipykernel_6078/4113958742.py:9: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd
2024-03-01 17:47:56.538648: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2024-03-01 17:47:56.592076: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-03-01 17:47:56.592103: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin 

## Data import and cleanup

In [3]:
csv_path = 'input.csv'

In [4]:
df = pd.read_csv(csv_path)
df = df.rename(columns={'Õhutemperatuur °C': 'temperature', 'Date Time': 'date', 'Tunni keskmine summaarne kiirgus W/m²' : 'radiation','Õhurõhk merepinna kõrgusel hPa' : 'pressure','Õhurõhk jaama kõrgusel hPa': 'pressure2','Tunni sademete summa mm' : 'precip','Suhteline õhuniiskus %' : 'humidity','Tunni miinimum õhutemperatuur °C' : 'tempmin','Tunni maksimum õhutemperatuur °C': 'tempmax','10 minuti keskmine tuule suund °' : 'winddir', '10 minuti keskmine tuule kiirus m/s' : 'windspeed', 'Tunni maksimum tuule kiirus m/s': 'windmax'})
# Slice [start:stop:step], starting from index 5 take every 6th record.
# df = df[5::6]

date_time = pd.to_datetime(df['Aasta'].astype(str) + df['Kuu'].astype(str).str.zfill(2) + df['Päev'].astype(str).str.zfill(2) + df['Kell (UTC)'], format='%Y%m%d%H:%M')
df = df.drop(['Aasta', 'Kuu', 'Päev', 'Kell (UTC)'], axis=1)

In [5]:
df.dropna(inplace=True)
df.describe().transpose()

,count,mean,std,min,25%,50%,75%,max
radiation,24.0,5.458333,11.053149,0.0,0.000,0.00,2.250,39.0
pressure,24.0,1014.712500,3.703149,1006.7,1012.275,1016.20,1017.275,1018.8
pressure2,24.0,1010.408333,3.679428,1002.4,1007.975,1011.80,1012.975,1014.4
precip,24.0,0.000000,0.000000,0.0,0.000,0.00,0.000,0.0
humidity,24.0,86.791667,5.356663,75.0,83.000,87.50,92.000,94.0
temperature,24.0,-6.562500,0.910238,-8.2,-7.150,-6.60,-6.100,-4.2
tempmin,24.0,-6.754167,0.957796,-8.6,-7.450,-6.75,-6.325,-4.2
tempmax,24.0,-6.233333,0.975200,-7.5,-6.925,-6.35,-5.725,-3.7
winddir,24.0,72.916667,63.799902,23.0,39.250,46.00,61.250,239.0
windspeed,24.0,3.200000,1.356466,0.6,2.275,3.00,4.325,5.3


In [6]:
wv = df['windspeed']
bad_wv = wv == -9999.0
wv[bad_wv] = 0.0

max_wv = df['windmax']
bad_max_wv = max_wv == -9999.0
max_wv[bad_max_wv] = 0.0

# The above inplace edits are reflected in the DataFrame.
df['windspeed'].min()

/tmp/ipykernel_55000/3044608421.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  wv[bad_wv] = 0.0
/tmp/ipykernel_55000/3044608421.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  max_wv[bad_max_wv] = 0.0


0.6

In [7]:
wv = df.pop('windspeed')
max_wv = df.pop('windmax')

# Convert to radians.
wd_rad = df.pop('winddir')*np.pi / 180

# Calculate the wind x and y components.
df['Wx'] = wv*np.cos(wd_rad)
df['Wy'] = wv*np.sin(wd_rad)

# Calculate the max wind x and y components.
df['max Wx'] = max_wv*np.cos(wd_rad)
df['max Wy'] = max_wv*np.sin(wd_rad)

In [8]:
timestamp_s = date_time.map(pd.Timestamp.timestamp)

In [9]:
day = 24*60*60
year = (365.2425)*day

df['Day sin'] = np.sin(timestamp_s * (2 * np.pi / day))
df['Day cos'] = np.cos(timestamp_s * (2 * np.pi / day))
df['Year sin'] = np.sin(timestamp_s * (2 * np.pi / year))
df['Year cos'] = np.cos(timestamp_s * (2 * np.pi / year))

### Load model

In [30]:
multi_lstm_model = tf.keras.models.load_model('multi_lstm_model.keras')
multi_lstm_model.summary()

mean = df.mean()
std = df.std()

#test_df = (df - mean) / std
test_df = df

reshaped_data = test_df.values.reshape(-1, 24, 16)

# Now you can use this reshaped data for prediction
predictions = multi_lstm_model.predict(reshaped_data)

Model: "sequential_17"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_11 (LSTM)              (None, 32)                6272      
                                                                 
 dense_23 (Dense)            (None, 384)               12672     
                                                                 
 reshape_6 (Reshape)         (None, 24, 16)            0         
                                                                 
Total params: 18944 (74.00 KB)
Trainable params: 18944 (74.00 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
1/1 [==============================] - 0s 249ms/step
[[[-1.34308791e+00  3.69086653e-01  3.72018188e-01  1.07792568e+00
    2.32991982e+00  2.59076297e-01  2.50733674e-01  2.77184576e-01
    8.92528772e-01 -4.25562590e-01  7.60815382e-01 -4.72578108e-01
    1.04827583e+00 -1.82950

In [ ]:
elem = ['Õhutemperatuur °C', 'Date Time', 'Tunni keskmine summaarne kiirgus W/m²','Õhurõhk merepinna kõrgusel hPa','Õhurõhk jaama kõrgusel hPa','Tunni sademete summa mm' ,'Suhteline õhuniiskus %' ,'Tunni miinimum õhutemperatuur °C' ,'Tunni maksimum õhutemperatuur °C','10 minuti keskmine tuule suund °', '10 minuti keskmine tuule kiirus m/s', 'Tunni maksimum tuule kiirus m/s', 'Day sin', 'Day cos', 'Year sin', 'Year cos']
andmed = {}
for f in range(16):
    andmed[elem[f]] = []
print(andmed)

for i in predictions[1]:
    for f in range(16):
        andmed[elem[f]].append(i[f])

In [1]:
temp = std.temperature*andmed['Õhutemperatuur °C'][23]+mean.temperature

rad = std.radiation*andmed['Tunni keskmine summaarne kiirgus W/m²'][23]+mean.radiation

sademed = std.precip*andmed['Tunni sademete summa mm'][23]+mean.precip

rõhk = std.pressure*andmed['Õhurõhk merepinna kõrgusel hPa'][23]+mean.pressure

wind_dir = (np.arcsin(andmed['Wy'][23]/wv[23])*180)/np.pi

NameError: name 'andmed' is not defined